In [1]:
import random
import csv
from quantumnet.components import Network, Controller

def run_simulation(clientes, num_requests, simulation_id):
    """
    Roda uma simulação e coleta métricas.

    Args:
        clientes (list): Lista de IDs dos clientes.
        num_requests (int): Número de requisições por rodada.
        simulation_id (int): ID da simulação atual.

    Returns:
        dict: Métricas coletadas (número de sucessos, total de EPRs usados, fidelidade média).
    """
    # Reinicia a rede e o controlador para cada simulação
    rede = Network()
    controller = Controller(rede)

    # Configura a topologia da rede
    rede.set_ready_topology('grade', 8, 4, 4, clients=clientes, server=10)

    # Gera requisições sem slices
    for _ in range(num_requests):
        alice_id = random.choice(clientes)
        bob_id = 10  # Servidor
        num_qubits = 10
        num_gates = 20
        scenario = 1

        # Gerar requisição
        request = rede.generate_request(
            alice_id=alice_id,
            bob_id=bob_id,
            num_qubits=num_qubits,
            num_gates=num_gates,
            scenario=scenario
        )

        # Enviar a requisição ao controlador
        controller.receive_request(request)

    # Processar e executar as requisições
    controller.process_requests()
    controller.send_scheduled_requests()

    # Coleta métricas
    schedule_report = controller.generate_schedule_report()
    total_used_eprs = rede.get_total_useds_eprs()
    avg_fidelity = rede.application_layer.avg_fidelity_on_applicationlayer()

    # Prepara as métricas para retorno
    metrics = {
        "simulation_id": simulation_id,
        "success_count": schedule_report.get("success", 0),
        "failure_count": schedule_report.get("failed", 0),
        "total_eprs_used": total_used_eprs,
        "average_fidelity": avg_fidelity,
    }
    return metrics


# Função para rodar múltiplas simulações e salvar os resultados em um arquivo CSV
def run_multiple_simulations(clientes, num_simulations, num_requests, output_file):
    """
    Roda múltiplas simulações e salva os resultados em um arquivo CSV.

    Args:
        clientes (list): Lista de IDs dos clientes.
        num_simulations (int): Número de simulações a executar.
        num_requests (int): Número de requisições por simulação.
        output_file (str): Nome do arquivo CSV para salvar os resultados.
    """
    with open(output_file, mode='w', newline='') as csvfile:
        fieldnames = ["simulation_id", "success_count", "failure_count", "total_eprs_used", "average_fidelity"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for i in range(num_simulations):
            print(f"Running simulation {i+1}/{num_simulations}...")
            metrics = run_simulation(clientes, num_requests, simulation_id=i+1)
            writer.writerow(metrics)
            print(f"Simulation {i+1} completed: {metrics}")


# Rodar múltiplas simulações
def main():
    clientes = [13, 8, 2 ,6,0,3,11,15]
    run_multiple_simulations(
        clientes=clientes,
        num_simulations=10,
        num_requests=10,
        output_file="simulation1random_results.csv"
    )


if __name__ == "__main__":
    main()


Running simulation 1/10...
Hosts inicializados
Canais inicializados
Pares EPRs adicionados
                      ┌───┐                    
q_0: ───────────────X─┤ H ├────────────────────
                    │ ├───┤     ┌───┐┌───┐     
q_1: ─X──────────X──┼─┤ S ├──■──┤ S ├┤ X ├─────
      │    ┌───┐ │  │ └───┘  │  └───┘└─┬─┘     
q_2: ─┼──■─┤ T ├─┼──┼────────┼─────────┼───────
      │  │ ├───┤ │  │        │         │       
q_3: ─┼──┼─┤ Y ├─┼──┼────────■─────────┼───────
      │  │ └───┘ │  │ ┌───┐            │       
q_4: ─┼──■───────┼──X─┤ H ├────────────┼───────
      │          │    └───┘            │  ┌───┐
q_5: ─┼──────────┼─────────────────────■──┤ S ├
      │          │         ┌───┐          └───┘
q_6: ─┼──────────┼──────X──┤ Y ├───────────────
      │          │      │  └───┘┌───┐     ┌───┐
q_7: ─┼──────────┼──X───┼────■──┤ Y ├──■──┤ S ├
      │          │  │   │    │  └───┘┌─┴─┐└───┘
q_8: ─┼──────────X──X───X────┼───────┤ X ├─────
      │                      │       └───┘   

In [ ]:
import random
import csv
from quantumnet.components import Network, Controller
from quantumnet.objects import Logger

# Função para rodar uma simulação com várias requisições
def run_simulation(protocol_name, num_requests, simulation_id):
    """
    Roda uma simulação e coleta métricas.

    Args:
        protocol_name (str): Nome do protocolo (e.g., "AC_BQC", "BFK_BQC").
        num_requests (int): Número de requisições por rodada.
        simulation_id (int): ID da simulação atual.

    Returns:
        dict: Métricas coletadas (número de sucessos, total de EPRs usados, fidelidade média).
    """
    # Reinicia a rede e o controlador para cada simulação
    rede = Network()
    controller = Controller(rede)

    # Configura a topologia da rede
    rede.set_ready_topology('grade', 8, 4, 4, clients=[13, 8, 2 ,6,0,3,11,15], server=10)

    clientes = [13, 8, 2 ,6,0,3,11,15]  # Clientes disponíveis

    for _ in range(num_requests):
        alice_id = random.choice(clientes)
        bob_id = 0  # Servidor
        num_qubits = 10
        num_gates = 20
        scenario = 1

        # Gerar requisição
        request = rede.generate_request(
            alice_id=alice_id,
            bob_id=bob_id,
            num_qubits=num_qubits,
            num_gates=num_gates,
            scenario=scenario,
        )

        request['protocol'] = protocol_name  

        # Enviar a requisição ao controlador
        controller.receive_request(request)

    # Processar e executar as requisições
    controller.process_requests()
    controller.send_scheduled_requests()

    # Coleta métricas
    schedule_report = controller.generate_schedule_report()
    total_used_eprs = rede.get_total_useds_eprs()
    avg_fidelity = rede.application_layer.avg_fidelity_on_applicationlayer()

    # Prepara as métricas para retorno
    metrics = {
        "simulation_id": simulation_id,
        "protocol": protocol_name,
        "success_count": schedule_report.get("success", 0),
        "failure_count": schedule_report.get("failed", 0),
        "total_eprs_used": total_used_eprs,
        "average_fidelity": avg_fidelity,
    }
    return metrics

# Função para rodar múltiplas simulações e salvar os resultados em um arquivo CSV
def run_multiple_simulations(num_simulations, num_requests, protocol_name, output_file):
    """
    Roda múltiplas simulações e salva os resultados em um arquivo CSV.

    Args:
        num_simulations (int): Número de simulações a executar.
        num_requests (int): Número de requisições por simulação.
        protocol_name (str): Nome do protocolo a ser usado.
        output_file (str): Nome do arquivo CSV para salvar os resultados.
    """
    with open(output_file, mode='w', newline='') as csvfile:
        fieldnames = ["simulation_id", "protocol", "success_count", "failure_count", "total_eprs_used", "average_fidelity"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for i in range(num_simulations):
            print(f"Running simulation {i+1}/{num_simulations} for protocol {protocol_name}...")
            metrics = run_simulation(protocol_name, num_requests, simulation_id=i+1)
            writer.writerow(metrics)
            print(f"Simulation {i+1} completed: {metrics}")

# Rodar múltiplas simulações
def main():
    run_multiple_simulations(
        num_simulations=10,
        num_requests=10,
        protocol_name="BFK_BQC",  # Escolha do protocolo (AC_BQC, BFK_BQC)
        output_file="simulation1BFK_results.csv"
    )

if __name__ == "__main__":
    main()


Running simulation 1/10 for protocol BFK_BQC...
Hosts inicializados
Canais inicializados
Pares EPRs adicionados
                                                        
q_0: ──────────────────────────────■───────■────────────
                                   │       │            
q_1: ──────────────────────────X───┼───────┼────■───────
                    ┌───┐┌───┐ │   │       │    │  ┌───┐
q_2: ───────X───────┤ X ├┤ X ├─┼───■───X───┼────┼──┤ Z ├
     ┌───┐  │  ┌───┐└─┬─┘├───┤ │ ┌───┐ │   │    │  └───┘
q_3: ┤ X ├──┼──┤ X ├──┼──┤ X ├─X─┤ X ├─┼───┼────┼───────
     ├───┤  │  ├───┤  │  └───┘   └─┬─┘ │   │    │       
q_4: ┤ T ├──┼──┤ Y ├──┼────────────┼───X───┼────┼───────
     ├───┤  │  └───┘  │            │       │    │       
q_5: ┤ Z ├──┼─────────■────────────┼───────┼────┼───────
     └───┘  │                      │       │    │       
q_6: ───────X──────────────────────■───────┼────┼───────
          ┌───┐┌───┐                       │    │       
q_7: ─────┤ Y ├┤ Y ├─────────────